This tutorial provides a simple and easy-to-understand guide for **beginner resource geologists** on how to use the *`pandas`*and `scipy` libraries in Python to find duplicates in drill hole data sets.

A simulated collars file, containing the coordinates of drill hole collars will be used to find  

* exact duplicates  
* partial duplicates
* and 'near' duplicates

In [60]:
# Import libraries
import numpy as np
import pandas as pd

# Load collars file (hosted on Github)
url = 'https://raw.githubusercontent.com/erebus-mre/Geostats_Code/main/00_Datasets/collars.csv'
collars = pd.read_csv(url)

# Have a look at the first few rows
display(collars.head())

,dhid,x,y,z
0,DH-1,0.0,0.0,89.0
1,DH-2,10.0,0.0,85.0
2,DH-3,20.0,0.0,89.0
3,DH-4,30.0,0.0,83.0
4,DH-5,40.0,0.0,88.0


# Exact Duplicates
Exact duplicates are rows that are identical in all columns.

These are easily identified using the [`duplicated()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) method in `pandas`:

In [61]:
# Find exact duplicates
dups = collars.duplicated(keep = False)

The method returns a series of boolean values, where `True` indicates that the row is one member of a duplicate pair.

In [62]:
dups.head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

This series can then be used to filter the original data frame to show only the duplicate rows.

In [63]:
duplicates = collars[dups].sort_values(by = 'dhid')
display(duplicates)

,dhid,x,y,z
99,DH-100,0.0,90.0,83.0
122,DH-100,0.0,90.0,83.0
112,DH-113,20.0,100.0,88.0
121,DH-113,20.0,100.0,88.0
30,DH-31,80.0,20.0,80.0
125,DH-31,80.0,20.0,80.0
51,DH-52,70.0,40.0,82.0
123,DH-52,70.0,40.0,82.0
81,DH-82,40.0,70.0,88.0
124,DH-82,40.0,70.0,88.0


The *keep* parameter determines which of the duplicates to mark as `True`. The default is `first`, which marks the first occurrence of the duplicate as `False` and the subsequent occurrences as `True`. Setting *keep* to `last` will mark the last occurrence as `False` and the previous occurrences as `True`.

This helps to remove **exact duplicates** from a data file:

In [64]:
collars_no_dups = collars[collars.duplicated(keep = 'first') == False]

The *pandas* method [`drop_duplicates()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.drop_duplicates.html#pandas.Series.drop_duplicates) will remove all the duplicates without the need to filter the data frame. 

In [65]:
collars = collars.drop_duplicates(keep = 'first')

# Partial Duplicates
Partial duplicates are rows that are identical in some columns but not all. These could occur when one or more of the coordinates in a record are changed without deleting the original record.

These holes will not be flagged as duplicated using the `duplicated()` method as implemented above. The **subset** parameter must be used to specify the columns to consider when identifying duplicates:

In the example below the *bhid*, *x*, and *y* values are the same but the *z* coordinate differs. Note the subset parameter: it is list of the columns that must be the same in two or more records for them to be considered duplicates.  

In [66]:
dups = collars.duplicated(subset = ['dhid','x','y'],keep = False)
duplicates = collars[dups].sort_values(by = 'dhid')
display(duplicates)

,dhid,x,y,z
101,DH-102,20.0,90.0,84.0
127,DH-102,20.0,90.0,85.0
11,DH-12,0.0,10.0,86.0
128,DH-12,0.0,10.0,87.0
12,DH-13,10.0,10.0,84.0
129,DH-13,10.0,10.0,85.0
32,DH-33,100.0,20.0,88.0
130,DH-33,100.0,20.0,89.0
82,DH-83,50.0,70.0,84.0
126,DH-83,50.0,70.0,85.0


The data frame can now be exported and the correct coordinates identified. Make sure you use the **keep = False** parameter as one does not know which of the duplicates is the correct one.

Remember to check all the possible combinations:  
* dhid, x and y are the same but z is different
* dhid, x and z are the same but y is different
* dhid, y and z are the same but x is different
* x,y and z are the same but dhid is different

Deleting the duplicate records are a bit more involved. I keep track of the index number of the erroneous records and then use the `.drop()` method to remove them from the original data frame.

For example, in the table above records with index numbers 101, 128, 12, 130 and 82 are wrong and must be removed: 

In [67]:
collars =collars.drop(index =[101, 128, 12, 130, 82])

# Confirm that the duplicates have been removed
dups = collars.duplicated(subset = ['dhid','x','y'],keep = False)
duplicates = collars[dups].sort_values(by = 'dhid')
display(duplicates)

,dhid,x,y,z
